In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

### CONNECTING TO DATABASES

In [2]:
# set up the function for conncting to the database 


def db_connect(hostname, username, password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = hostname,
            user = username,
            passwd = password,
            database = db_name
        )
        print(f'Connected to database {db_name}')
    except Error as err:
        print(f'Error: {err}')
    return connection

connection = db_connect('localhost','root','1234','northwind')

Connected to database northwind


In [9]:
# create a readn query function

def read_query(connection, query):
    result = None
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
    except Error as err:
        print(f'Error: {err}')
    return result

# call the read_query function

query_1 = '''
select o.orderid, o.shipcountry,od.unitprice,od.quantity,
    concat_ws(' ',e.firstname,e.lastname) as fullname,
    o.shippeddate,o.requireddate,s.companyname,c.contactname,
    o.freight
from orders o
join `order details` od on od.orderid = o.orderid
join employees e on e.employeeid = o.employeeid
join shippers s on s.shipperid = o.shipvia
join customers c on c.customerid = o.customerid
order by freight desc;

'''
results = read_query(connection, query_1)

data_list = []

for row in results:
    row = list(row)
    data_list.append(row)


columns = ['orderid','country','price','qty',
           'fullname','shipdate','delivery','shipper','customer','freight']
orders_df = pd.DataFrame(data=data_list, columns=columns)

## Basic Pandas Method

In [12]:
# head method
orders_df.head(3)

,orderid,country,price,qty,fullname,shipdate,delivery,shipper,customer,freight
0,10540,Germany,10.0000,60,Janet Leverling,1997-06-13,1997-06-16,Federal Shipping,Horst Kloss,1007.6400
1,10540,Germany,31.2300,40,Janet Leverling,1997-06-13,1997-06-16,Federal Shipping,Horst Kloss,1007.6400
2,10540,Germany,263.5000,30,Janet Leverling,1997-06-13,1997-06-16,Federal Shipping,Horst Kloss,1007.6400


In [15]:
# .tail method

orders_df.tail(3)

,orderid,country,price,qty,fullname,shipdate,delivery,shipper,customer,freight
2152,10296,Venezuela,28.8000,15,Michael Suyama,1996-09-11,1996-10-01,Speedy Express,Carlos Gonzlez,0.1200
2153,10972,France,39.0000,6,Margaret Peacock,1998-03-26,1998-04-21,United Package,Daniel Tonini,0.0200
2154,10972,France,2.5000,7,Margaret Peacock,1998-03-26,1998-04-21,United Package,Daniel Tonini,0.0200


In [19]:
# shape attribute 

orders_df.shape

(2155, 10)

In [23]:
# info method 

orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   orderid   2155 non-null   object        
 1   country   2155 non-null   object        
 2   price     2155 non-null   float64       
 3   qty       2155 non-null   int64         
 4   fullname  2155 non-null   object        
 5   shipdate  2082 non-null   datetime64[ns]
 6   delivery  2155 non-null   datetime64[ns]
 7   shipper   2155 non-null   object        
 8   customer  2155 non-null   object        
 9   freight   2155 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 168.5+ KB


In [22]:
# convert data type with astype method


orders_df = orders_df.astype(
    {'orderid':'str',
     'price':'float',
     'freight':'float'}
)


In [31]:
# checking null values with isnull method

orders_df.isnull().sum()

orderid     0
country     0
price       0
qty         0
fullname    0
shipdate    0
delivery    0
shipper     0
customer    0
freight     0
dtype: int64

In [30]:
# handling missing values dropna, fillna

orders_df.dropna(inplace=True)

In [34]:
# reset index method

orders_df.reset_index(inplace=True, drop=True)

In [37]:
# .drop method

orders_df.drop(columns=['index'], inplace=True)

In [41]:
orders_df.drop(list(range(0,100)))

,orderid,country,price,qty,fullname,shipdate,delivery,shipper,customer,freight
100,10979,Austria,4.5,80,Laura Callahan,1998-03-31,1998-04-23,United Package,Roland Mendel,353.07
101,10979,Austria,43.9,30,Laura Callahan,1998-03-31,1998-04-23,United Package,Roland Mendel,353.07
102,10979,Austria,12.5,24,Laura Callahan,1998-03-31,1998-04-23,United Package,Roland Mendel,353.07
103,10979,Austria,43.9,35,Laura Callahan,1998-03-31,1998-04-23,United Package,Roland Mendel,353.07
104,10657,USA,15.5,50,Andrew Fuller,1997-09-15,1997-10-02,United Package,Jose Pavarotti,352.69
...,...,...,...,...,...,...,...,...,...,...
2077,10296,Venezuela,16.8,12,Michael Suyama,1996-09-11,1996-10-01,Speedy Express,Carlos Gonzlez,0.12
2078,10296,Venezuela,13.9,30,Michael Suyama,1996-09-11,1996-10-01,Speedy Express,Carlos Gonzlez,0.12
2079,10296,Venezuela,28.8,15,Michael Suyama,1996-09-11,1996-10-01,Speedy Express,Carlos Gonzlez,0.12
2080,10972,France,39.0,6,Margaret Peacock,1998-03-26,1998-04-21,United Package,Daniel Tonini,0.02


In [ ]:
empty = list(range(0,100))
empty